Семинар
1 пункт

In [11]:
import tensorflow as tf

def transformation_network(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    
    outputs = tf.keras.layers.Dense(2)(x)  
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model
input_shape = (28, 28, 1)  
model = transformation_network(input_shape)
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0   

2 пункт

In [3]:
import cv2


image = cv2.imread('image.jpg')


object1_x, object1_y = 100, 150
object2_x, object2_y = 200, 250


if object1_x < object2_x:
   print('Object 1 is to the left of Object 2')
elif object1_x > object2_x:
   print('Object 1 is to the right of Object 2')
else:
   print('Object 1 and Object 2 are in the same column')

if object1_y < object2_y:
    print('Object 1 is above Object 2')
elif (object1_y > object2_y):
    print('Object 1 is below Object 2')
else:
  print('Object 1 and Object 2 are in the same row')

Object 1 is to the left of Object 2
Object 1 is above Object 2


3 пункт

In [17]:

"""CLEVR dataset."""

import collections
import json
import os

from etils import epath
import numpy as np
from tensorflow_datasets.core.utils.lazy_imports_utils import tensorflow as tf
import tensorflow_datasets.public_api as tfds

_BASE_URL = "https://cs.stanford.edu/people/jcjohns/clevr/"
_DOWNLOAD_URL = "https://dl.fbaipublicfiles.com/clevr/CLEVR_v1.0.zip"


class Build(tfds.core.GeneratorBasedBuilder):
  """CLEVR dataset."""

  VERSION = tfds.core.Version("3.1.0")
  SUPPORTED_VERSIONS = [
      tfds.core.Version("3.0.0"),
  ]
  RELEASE_NOTES = {
      "3.1.0": "Add question/answer text.",
  }

  def _info(self):
    features = {
        "image": tfds.features.Image(),
        "file_name": tfds.features.Text(),
        "objects": tfds.features.Sequence({
            "color": tfds.features.ClassLabel(
                names=[
                    "gray",
                    "blue",
                    "brown",
                    "yellow",
                    "red",
                    "green",
                    "purple",
                    "cyan",
                ]
            ),
            "material": tfds.features.ClassLabel(names=["rubber", "metal"]),
            "shape": tfds.features.ClassLabel(
                names=["cube", "sphere", "cylinder"]
            ),
            "size": tfds.features.ClassLabel(names=["small", "large"]),
            "rotation": tfds.features.Tensor(shape=(), dtype=np.float32),
            "3d_coords": tfds.features.Tensor(shape=(3,), dtype=np.float32),
            "pixel_coords": tfds.features.Tensor(shape=(3,), dtype=np.float32),
        }),
    }
    if self.version > "3.0.0":
      features["question_answer"] = tfds.features.Sequence({
          "question": tfds.features.Text(),
          "answer": tfds.features.Text(),
      })
    return self.dataset_info_from_configs(
        features=tfds.features.FeaturesDict(features),
        homepage=_BASE_URL,
    )

  def _split_generators(self, dl_manager):
    """Returns splits."""
    path = dl_manager.download_and_extract(_DOWNLOAD_URL)

    images_path_dir = os.path.join(path, "CLEVR_v1.0/images")
    questions_path_dir = os.path.join(path, "CLEVR_v1.0/questions")
    scenes_path_dir = os.path.join(path, "CLEVR_v1.0/scenes")

    splits = []
    for split_name in ["train", "val", "test"]:
      name_map = {
          "train": tfds.Split.TRAIN,
          "val": tfds.Split.VALIDATION,
          "test": tfds.Split.TEST,
      }
      splits.append(
          tfds.core.SplitGenerator(
              name=name_map[split_name],
              gen_kwargs={
                  "images_dir_path": os.path.join(images_path_dir, split_name),
                  "question_file": os.path.join(
                      questions_path_dir,
                      "CLEVR_{}_questions.json".format(split_name),
                  ),
                  "scenes_description_file": os.path.join(
                      scenes_path_dir, "CLEVR_{}_scenes.json".format(split_name)
                  ),
              },
          )
      )

    return splits

  def _generate_examples(
      self, images_dir_path, question_file, scenes_description_file
  ):
    image_paths = sorted(
        [
            os.path.join(images_dir_path, filename)
            for filename in tf.io.gfile.listdir(images_dir_path)
        ]
    )

    with epath.Path(question_file).open() as f:
      questions_json = json.load(f)
    questions = collections.defaultdict(list)
    for q in questions_json["questions"]:
      questions[q["image_filename"]].append({
          "question": q["question"],
          "answer": q.get("answer", ""),  
      })

    if tf.io.gfile.exists(scenes_description_file):
      with epath.Path(scenes_description_file).open() as f:
        scenes_json = json.load(f)
    else:
      scenes_json = {"scenes": [{"objects": []}] * len(image_paths)}

    attrs = [
        "color",
        "material",
        "shape",
        "size",
        "rotation",
        "pixel_coords",
        "3d_coords",
    ]
    for image_path, scene in zip(image_paths, scenes_json["scenes"]):
      objects = scene["objects"]
      fname = os.path.basename(image_path)
      record = {
          "image": image_path,
          "file_name": fname,
          "question_answer": questions[fname],
          "objects": [{attr: obj[attr] for attr in attrs} for obj in objects],  # pylint: disable=g-complex-comprehension
      }
      yield fname, record